In [2]:
import pandas as pd
import numpy as np
import os
from PIL import Image
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras.preprocessing.image import img_to_array
import cv2

In [3]:
import dataclasses
import zipfile as zf

dataset = !wget https://storage.googleapis.com/new_music_bucket/spectrograms.zip

dataset = zf.ZipFile(f'spectrograms.zip', 'r')
dataset.extractall()
dataset.close()

In [4]:
merged_df = pd.read_csv('./DF/merged_df.csv')

In [5]:
shuffled_df = merged_df.sample(frac=1)
train_df, val_df, test_df = shuffled_df[:7000], shuffled_df[7000:8550], shuffled_df[8550:]

In [6]:
def get_X_y(df):

  X_pic, X_stats = [], []
  y = []

  for name in df['NPZ_Path']:
    loaded_npz = np.load(name, allow_pickle=True)

    pic = loaded_npz['pic']
    X_pic.append(pic)

    stats = loaded_npz['stats']
    X_stats.append(stats)
    
    
    y.append(loaded_npz['track_popularity'])

  X_pic, X_stats = np.array(X_pic), np.array(X_stats)
  y = np.array(y)

  return (X_pic, X_stats), y



In [7]:
# Get the training data
(X_train_pic, X_train_stats), y_train = get_X_y(train_df)

In [8]:
X_train_pic.shape

(7000, 240, 320, 3)

In [ ]:
# Get the validation data

(X_val_pic, X_val_stats), y_val = get_X_y(val_df)

(X_val_pic.shape, X_val_stats.shape), y_val.shape

(((1550, 240, 320, 3), (1550, 11)), (1550,))

In [ ]:
# %% Get the test data
(X_test_pic, X_test_stats), y_test = get_X_y(test_df)

(X_test_pic.shape, X_test_stats.shape), y_test.shape

(((1605, 240, 320, 3), (1605, 11)), (1605,))

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
X_train_stats_scaled = scaler.fit_transform(X_train_stats)
X_test_stats_scaled = scaler.fit_transform(X_test_stats)
X_val_stats_scaled = scaler.fit_transform(X_val_stats)

In [ ]:
# Define the Model

from tensorflow.keras import layers
from tensorflow.keras.models import Model
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input

# Define the Picture (CNN) Stream

input_pic = layers.Input(shape=(240, 320, 3))

x         = MobileNetV2(input_shape=((240, 320, 3)), include_top=False)(input_pic)
x         = layers.Conv2D(128, (3, 3), activation='relu', padding='same', input_shape=(240, 320, 3))(x)
x         = layers.Conv2D(64, (3, 3), activation='relu', padding='same', input_shape=(240, 320, 3))(x)
x         = layers.GlobalAveragePooling2D()(x)
x         = layers.Dense(10, activation = 'sigmoid')(x)
x         = layers.Dense(10, activation = 'sigmoid')(x)
x         = Model(inputs=input_pic, outputs=x)

# Define the Stats (Feed-Forward) Stream

input_stats = layers.Input(shape=(11,))
y = layers.Dense(64, kernel_regularizer='L1L2')(input_stats)
y = layers.Flatten()(y) 
y = layers.Dense(32, activation="relu", kernel_regularizer='L1L2')(y)
y = layers.Dense(10, activation="relu")(y)
y = Model(inputs=input_stats, outputs=y)


# Concatenate the two streams together
combined = layers.concatenate([x.output, y.output])

# Define joined Feed-Forward Layer
z = layers.Dense(4, activation="relu")(combined)

# Define output node of 1 linear neuron (regression task)
z = layers.Dense(1, activation="linear")(z)


# Define the final model
model = Model(inputs=[x.input, y.input], outputs=z)

In [ ]:
from tensorflow.keras.optimizers import Adam

optimizer = Adam(learning_rate=0.01)

model.compile(loss='mse', optimizer=optimizer, metrics=['mean_absolute_error'])

In [ ]:
evaluation_list = []

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint

cp = ModelCheckpoint('model/', save_best_only=True)

model.fit(x=[X_train_pic, X_train_stats_scaled], y=y_train, validation_data=([X_val_pic, X_val_stats_scaled], y_val), epochs=5, callbacks=[cp])

Epoch 1/5
219/219 [==============================] - ETA: 0s - loss: 404.5878 - mean_absolute_error: 15.6831

219/219 [==============================] - 109s 437ms/step - loss: 404.5878 - mean_absolute_error: 15.6831 - val_loss: 307.6714 - val_mean_absolute_error: 14.3558
Epoch 2/5
219/219 [==============================] - ETA: 0s - loss: 278.4871 - mean_absolute_error: 13.4800

219/219 [==============================] - 95s 433ms/step - loss: 278.4871 - mean_absolute_error: 13.4800 - val_loss: 283.0169 - val_mean_absolute_error: 13.6886
Epoch 3/5
219/219 [==============================] - 75s 342ms/step - loss: 278.0450 - mean_absolute_error: 13.4585 - val_loss: 338.7612 - val_mean_absolute_error: 15.1124
Epoch 4/5
219/219 [==============================] - 74s 338ms/step - loss: 276.1990 - mean_absolute_error: 13.4351 - val_loss: 303.2436 - val_mean_absolute_error: 14.2566
Epoch 5/5
219/219 [==============================] - 75s 340ms/step - loss: 273.2068 - mean_absolute_error: 13.3733 - val_loss: 283.7263 - val_mean_absolute_error: 13.7176


In [ ]:
evaluation_list.append(model.evaluate((X_test_pic, X_test_stats_scaled), y_test))

51/51 [==============================] - 4s 70ms/step - loss: 262.4462 - mean_absolute_error: 13.0471


In [ ]:
evaluation_list

[[262.4461669921875, 13.047080993652344]]

In [ ]:
model.save('./my_model') 